In [1]:
from torchvision.datasets import MNIST, mnist, CIFAR10
from torchvision import transforms

In [2]:
import torch.nn.functional as F

In [3]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.distributions import Categorical
from torch.utils.data import DataLoader

In [4]:
from tqdm.notebook import tqdm_notebook

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns

/home/alex1um/.local/lib/python3.11/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [6]:
from itertools import chain
import pandas as pd

In [7]:
from torch.utils.tensorboard import SummaryWriter
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [41]:
class CustomTargetTransform:
    def __init__(self, num_classes=10):
        self.num_classes = num_classes

    def __call__(self, target):
        new_target = torch.zeros(self.num_classes, dtype=torch.float, device=device)
        new_target[target] = 1
        return new_target

transform = transforms.Compose([
    transforms.PILToTensor(),
    transforms.Lambda(lambda x: x.float().to(device))
])

# data_loader = DataLoader(dataset, batch_size=800, shuffle=True)

In [43]:
# dataset = mnist.FashionMNIST("data", download=True, train=True, transform=transform, target_transform=CustomTargetTransform())
# dataset_target = mnist.FashionMNIST("data", download=True, train=False, transform=transforms.PILToTensor())
dataset = CIFAR10("data", download=True, train=True, transform=transform, target_transform=CustomTargetTransform())
dataset_target = CIFAR10("data", download=True, train=False, transform=transform, target_transform=CustomTargetTransform())

Files already downloaded and verified
Files already downloaded and verified


In [46]:
target_loader = DataLoader(dataset_target, batch_size=10000, shuffle=False)

In [11]:
class ResBlock(nn.Module):
    def __init__(self, in_channels, batch_norm1, batch_norm2, inter_channels, kernel_size1, kernel_size2, stride1, stride2, padding1=1, padding2=1, in_size=28):
        super().__init__()
        block = [
            nn.Conv2d(in_channels, inter_channels, kernel_size1, stride1, padding1),
        ]
        if batch_norm1:
            block.append(nn.BatchNorm2d(inter_channels))
        block.append(nn.ReLU())
        block.append(nn.Conv2d(inter_channels, in_channels, kernel_size2, stride2, padding2))
        if batch_norm2:
            block.append(nn.BatchNorm2d(in_channels))
        self.block = nn.Sequential(
            *block
        )
    
    def get_out_size(self, in_size):
        for layer in (self.block[0], self.block[-1] if type(self.block[-1]) == nn.Conv2d else self.block[-2]):
            in_size = (in_size - layer.kernel_size[0] + 2 * layer.padding[0]) // layer.stride[0] + 1
        return in_size

    def forward(self, x):
        return F.relu(x + self.block(x))


In [59]:
def get_name(model: nn.Module) -> list[nn.Module]:
    modules_counter = {}
    for m in model.modules():
        name = m._get_name()
        if name not in modules_counter:
            modules_counter[name] = 0
        modules_counter[name] += 1
    return "_".join(f'{k}-{v}' for k, v in modules_counter.items())

In [57]:
def create_model(
        img_size,
        pool_kernel_size,
        pool_stride,
        *layers,
        linears: list[int],
        lr=0.01,
        batch_size=800,
        epoch_count=10,
        softmax=True,
        in_channels=1,
        maxpool=True,
        avgpool=False,
    ) -> tuple[nn.Sequential, nn.CrossEntropyLoss, torch.optim.SGD, SummaryWriter]:
    torch.manual_seed(0)
    blocks = []
    outs = img_size
    for layer in layers:
        blocks.append(
            layer
        )
        tpe = type(layer)
        if tpe == ResBlock:
            outs = layer.get_out_size(outs)
        elif tpe == nn.Conv2d:
            outs = (outs - layer.kernel_size[0] + 2 * layer.padding[0]) // layer.stride[0] + 1
            in_channels = layer.out_channels
        elif tpe == nn.MaxPool2d:
            outs = (outs - layer.kernel_size) // layer.stride + 1
    if maxpool:
        outs = (outs - pool_kernel_size) // pool_stride + 1
        if avgpool:
            blocks.append(nn.AvgPool2d(kernel_size=pool_kernel_size, stride=pool_stride))
        else:
            blocks.append(nn.MaxPool2d(kernel_size=pool_kernel_size, stride=pool_stride))
    outs = outs * outs * in_channels
    blocks.append(nn.Flatten(1))
    for layer in linears:
        blocks.append(nn.Linear(outs, layer))
        blocks.append(nn.ReLU())
        outs = layer
    blocks.append(nn.Linear(outs, 10))
    if softmax:
        blocks.append(nn.Softmax())
    model = nn.Sequential(
        *blocks,
    ).to(device)
    er_f = nn.CrossEntropyLoss()
    optim = torch.optim.AdamW(model.parameters(), lr=lr)
    # name = "_".join(m._get_name() for m in model.modules())
    # name = f"_{len(blocks_out_channels)}_{lr}_{blocks_kernel_size}_{blocks_stride}_{pool_kernel_size}_{pool_stride}"
    name = get_name(model)
    print(name)
    writer = SummaryWriter(comment=name)
    return model, er_f, optim, writer, name, batch_size, epoch_count


In [48]:
def write_summary(writer: SummaryWriter, model: nn.Module):
    param_dict = {f"layer_{i}": str(m) for m, i in enumerate(get_name(model))}
    param_table = pd.DataFrame(param_dict, index=[0])
    for target_data, target_labels in target_loader:
        target_labels = target_labels.max(1).indices
    predicted = model(target_data)
    predicted_labels = predicted.max(1).indices
    confusion_matrix = torch.zeros(10, 10, device=device)
    for i in range(len(predicted_labels)):
        confusion_matrix[predicted_labels[i].long()][target_labels[i].long()] += 1
    confusion_matrix = confusion_matrix.cpu()
    fig = plt.gcf()
    fig.clear()
    ax = fig.add_subplot(111)
    hist = predicted_labels[predicted_labels == target_labels]
    error_hist = target_labels[predicted_labels != target_labels]
    percent: torch.tensor = ((predicted_labels == target_labels).sum() / len(target_labels))
    sns.heatmap(confusion_matrix, annot=True, fmt="g", ax=ax)
    writer.add_figure("confusion_matrix", fig)
    writer.add_scalar("accuracy", percent)
    fig.clear()
    ax = fig.add_subplot(111)
    sns.histplot(hist.cpu(), stat="count", discrete=True, bins=range(10), ax=ax)
    writer.add_figure("right hist", fig)
    fig.clear()
    ax = fig.add_subplot(111)
    sns.histplot(error_hist.cpu(), stat="count", discrete=True, bins=range(10), ax=ax)
    writer.add_figure("error hist", fig)
    writer.add_text("param_table", param_table.to_markdown())
    writer.add_text("accuracy", str(percent.item()))

In [63]:
models = (
    # create_model(
    #     32,
    #     2,
    #     2,
    #     nn.Conv2d(3, 6, 5),
    #     nn.MaxPool2d(2, 2),
    #     nn.Conv2d(6, 16, 5),
    #     linears=[],
    #     lr=0.0001,
    #     batch_size=30,
    #     epoch_count=1,
    # ), # 81 - 1000
    # create_model(
    #     32,
    #     2,
    #     2,
    #     nn.Conv2d(3, 6, 5),
    #     nn.ReLU(),
    #     ResBlock(6, True, True, 10, 3, 3, 1, 1, in_size=32),
    #     nn.MaxPool2d(2, 2),
    #     nn.Conv2d(6, 16, 5),
    #     nn.ReLU(),
    #     ResBlock(16, True, True, 20, 3, 3, 1, 1, in_size=32),
    #     nn.MaxPool2d(2, 2),
    #     linears=[64, 32],
    #     lr=0.001,
    #     batch_size=4,
    #     epoch_count=1,
    #     maxpool=False,
    #     softmax=False,
    # ), # 81 - 1000
    # create_model(
    #     32,
    #     2,
    #     2,
    #     ResBlock(3, True, True, 10, 3, 3, 1, 1, in_size=32),
    #     nn.MaxPool2d(2, 2),
    #     nn.ReLU(),
    #     nn.Conv2d(3, 10, 3, 3),
    #     ResBlock(10, True, True, 20, 3, 3, 1, 1, in_size=32),
    #     nn.ReLU(),
    #     linears=[64, 32],
    #     lr=0.001,
    #     batch_size=20,
    #     epoch_count=1,
    #     maxpool=True,
    #     softmax=True,
    # ), # 81 - 1000
    create_model(
        32,
        2,
        2,
        nn.Conv2d(3, 8, 2, 2),
        nn.MaxPool2d(2, 2),
        nn.BatchNorm2d(8),
        nn.ReLU(),
        ResBlock(8, False, False, 8, 3, 3, 1, 1, in_size=32),
        nn.ReLU(),
        ResBlock(8, False, False, 1, 3, 3, 1, 1, in_size=32),
        nn.ReLU(),
        ResBlock(8, False, False, 8, 3, 3, 1, 1, in_size=32),
        nn.ReLU(),
        nn.Conv2d(8, 16, 2, 2),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        ResBlock(16, False, False, 16, 3, 3, 1, 1, in_size=32),
        nn.ReLU(),
        ResBlock(16, False, False, 1, 3, 3, 1, 1, in_size=32),
        nn.ReLU(),
        ResBlock(16, False, False, 16, 3, 3, 1, 1, in_size=32),
        nn.ReLU(),
        nn.Conv2d(16, 32, 2, 2),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        ResBlock(32, False, False, 32, 3, 3, 1, 1, in_size=32),
        nn.ReLU(),
        ResBlock(32, False, False, 1, 3, 3, 1, 1, in_size=32),
        nn.ReLU(),
        ResBlock(32, False, False, 32, 3, 3, 1, 1, in_size=32),
        nn.ReLU(),
        linears=[],
        lr=0.01,
        batch_size=20,
        epoch_count=1,
        maxpool=True,
        avgpool=True,
        softmax=False,
    ), # 81 - 1000
)

Sequential-10_Conv2d-21_MaxPool2d-1_BatchNorm2d-3_ReLU-21_ResBlock-9_AvgPool2d-1_Flatten-1_Linear-1


In [64]:
try:
    for model, er_f, optim, writer, name, batch_count, epoch_count in tqdm_notebook(models):
        #  = create_model(28, *param)
        torch.manual_seed(0)
        data_loader = DataLoader(dataset, batch_size=batch_count, shuffle=True)
        total_epochs = 0
        for epoch in tqdm_notebook(range(epoch_count)):
            for image, target in tqdm_notebook(data_loader, leave=False):
                optim.zero_grad()
                outs = model(image)
                loss = er_f(outs, target)
                loss.backward()
                optim.step()
                percent: torch.tensor = ((outs.max(1).indices == target.max(1).indices).sum() / len(target))
                writer.add_scalar("loss", loss, total_epochs)
                writer.add_scalar("train_accuracy", percent, total_epochs)
                for target_data, target_labels in target_loader:
                    predicted = model(target_data)
                    percent: torch.tensor = ((predicted.max(1).indices == target_labels.max(1).indices).sum() / len(target_labels))
                    writer.add_scalar("test_accuracy", percent, total_epochs)
                total_epochs += 1
finally:
    write_summary(writer, model)
    torch.save(model, f"model_{repr(dataset.__class__).split('.')[-1][:-2]}_{percent:.2f}.pth")

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2500 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [76]:
optim = torch.optim.AdamW(model.parameters(), lr=0.0001)
data_loader = DataLoader(dataset, batch_size=100, shuffle=True)
epoches = 1

In [64]:
optim = torch.optim.AdamW(model.parameters(), lr=0.001)
data_loader = DataLoader(dataset, batch_size=1000, shuffle=True)
epoches = 10

In [22]:
data_loader = DataLoader(dataset, batch_size=500, shuffle=True)
epoches = 10

In [50]:
try:
    for epoch in tqdm_notebook(range(epoches)):
        for image, target in tqdm_notebook(data_loader, leave=False):
            optim.zero_grad()
            outs = model(image)
            loss = er_f(outs, target)
            loss.backward()
            optim.step()
            percent: torch.tensor = ((outs.max(1).indices == target.max(1).indices).sum() / len(target))
            writer.add_scalar("loss", loss, total_epochs)
            writer.add_scalar("train_accuracy", percent, total_epochs)
            for target_data, target_labels in target_loader:
                predicted = model(target_data)
                percent: torch.tensor = ((predicted.max(1).indices == target_labels.max(1).indices).sum() / len(target_labels))
                writer.add_scalar("test_accuracy", percent, total_epochs)
            total_epochs += 1
finally:
    write_summary(writer, model)
    torch.save(model, f"model_{repr(dataset.__class__).split('.')[-1][:-2]}_{percent:.2f}.pth")

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

tensor([[[[158., 159., 165.,  ..., 137., 126., 116.],
          [152., 151., 159.,  ..., 136., 125., 119.],
          [151., 151., 158.,  ..., 139., 130., 120.],
          ...,
          [ 68.,  42.,  31.,  ...,  38.,  13.,  40.],
          [ 61.,  49.,  35.,  ...,  26.,  29.,  20.],
          [ 54.,  56.,  45.,  ...,  24.,  34.,  21.]],

         [[112., 111., 116.,  ...,  95.,  91.,  85.],
          [112., 110., 114.,  ...,  95.,  91.,  88.],
          [110., 109., 111.,  ...,  98.,  95.,  89.],
          ...,
          [124., 100.,  88.,  ...,  97.,  64.,  85.],
          [116., 102.,  85.,  ...,  82.,  82.,  64.],
          [107., 105.,  89.,  ...,  77.,  84.,  67.]],

         [[ 49.,  47.,  51.,  ...,  36.,  36.,  33.],
          [ 51.,  40.,  45.,  ...,  31.,  32.,  34.],
          [ 47.,  33.,  36.,  ...,  34.,  34.,  33.],
          ...,
          [177., 148., 137.,  ..., 146., 108., 127.],
          [168., 148., 132.,  ..., 130., 126., 107.],
          [160., 149., 132.,  ...

KeyboardInterrupt: 

In [27]:
img, target = next(iter(data_loader))

In [30]:
target

tensor([[0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')

In [ ]:
torch.load(model, f"model_{repr(dataset.__class__).split('.')[-1][:-2]}_{name:.2f}.pth")